In [ ]:
def clean_and_preprocess_csv(csv_file):
    
    lines = []
    try:
        with open(csv_file, 'r') as file:
            lines = file.readlines()
    except Exception as e:
        print(f"Error reading file: {str(e)}")
        return None
    
    if not lines:
        print("Empty file or file could not be read")
        return None
    
    
    header = lines[0].strip().split(',')
    columns = {name: idx for idx, name in enumerate(header)}
    
    
    cleaned_data = [header]
    
    
    for i in range(1, len(lines)):
        row = lines[i].strip().split(',')
        
        # Skip empty rows
        if not row or all(cell == '' for cell in row):
            continue
        
        # Handle missing values and ensure row has correct length
        while len(row) < len(header):
            row.append('')
        
        # Data type conversion and validation
        for col_name, col_idx in columns.items():
            if col_idx < len(row):
                
                if col_name == 'CLAIM_AMOUNT' or col_name == 'PREMIUM_COLLECTED' or col_name == 'PAID_AMOUNT':
                    try:
                        
                        if row[col_idx]:
                            row[col_idx] = float(row[col_idx])
                    except ValueError:
                        
                        row[col_idx] = ''
                        
        
        cleaned_data.append(row)
    
    return cleaned_data

def get_city_statistics(data):
    """Calculate statistics for each city to determine which one to close"""
    if not data or len(data) <= 1:
        return None
    
    
    header = data[0]
    columns = {name: idx for idx, name in enumerate(header)}
    
    
    city_stats = {}
    
    
    for i in range(1, len(data)):
        row = data[i]
        
        
        city_idx = columns.get('CITY')
        city = row[city_idx] if city_idx is not None and city_idx < len(row) else ''
        
        if not city:
            continue  
        
        
        if city not in city_stats:
            city_stats[city] = {
                'total_claims': 0,
                'total_claim_amount': 0,
                'total_premium': 0,
                'total_paid': 0,
                'rejected_claims': 0
            }
        
        
        city_stats[city]['total_claims'] += 1
        
        
        claim_idx = columns.get('CLAIM_AMOUNT')
        if claim_idx is not None and claim_idx < len(row) and row[claim_idx] != '':
            city_stats[city]['total_claim_amount'] += float(row[claim_idx])
        
        
        premium_idx = columns.get('PREMIUM_COLLECTED')
        if premium_idx is not None and premium_idx < len(row) and row[premium_idx] != '':
            city_stats[city]['total_premium'] += float(row[premium_idx])
        
        
        paid_idx = columns.get('PAID_AMOUNT')
        if paid_idx is not None and paid_idx < len(row) and row[paid_idx] != '':
            paid_amount = float(row[paid_idx])
            city_stats[city]['total_paid'] += paid_amount
            if paid_amount == 0:  
                city_stats[city]['rejected_claims'] += 1
    
    # Calculate derived metrics for each city
    for city, stats in city_stats.items():
        # Calculate rejection rate
        if stats['total_claims'] > 0:
            stats['rejection_rate'] = stats['rejected_claims'] / stats['total_claims']
        else:
            stats['rejection_rate'] = 0
            
        
        stats['profit'] = stats['total_premium'] - stats['total_paid']
        
        
        if stats['total_premium'] > 0:
            stats['loss_ratio'] = stats['total_paid'] / stats['total_premium']
        else:
            stats['loss_ratio'] = 0
    
    return city_stats

def recommend_city_closure(city_stats):
    """Recommend which city to close based on various metrics"""
    if not city_stats:
        return None
    
    # Filter to only consider the four cities mentioned in the requirements
    target_cities = ['PUNE', 'KOLKATA', 'RANCHI', 'GUWAHATI']
    filtered_stats = {city: stats for city, stats in city_stats.items() if city in target_cities}
    
    if not filtered_stats:
        return None
    
    
    city_scores = {}
    
    # Score based on profitability (lower profit gets lower score)
    profits = [(city, stats['profit']) for city, stats in filtered_stats.items()]
    profits.sort(key=lambda x: x[1])
    for i, (city, _) in enumerate(profits):
        city_scores[city] = city_scores.get(city, 0) + i
    
    # Score based on loss ratio (higher ratio gets lower score)
    loss_ratios = [(city, stats['loss_ratio']) for city, stats in filtered_stats.items()]
    loss_ratios.sort(key=lambda x: x[1], reverse=True)
    for i, (city, _) in enumerate(loss_ratios):
        city_scores[city] = city_scores.get(city, 0) + i
    
    # Score based on total claims (lower claims gets lower score - less business)
    claims = [(city, stats['total_claims']) for city, stats in filtered_stats.items()]
    claims.sort(key=lambda x: x[1])
    for i, (city, _) in enumerate(claims):
        city_scores[city] = city_scores.get(city, 0) + i
    
    
    recommendation = min(city_scores.items(), key=lambda x: x[1])
    
    return recommendation[0], filtered_stats

# Fixed function to handle errors properly
def handle_error(error_message):
    print(f"Error: {error_message}")
    return "Error"

# Fixed function to check if a reason exists in rejection text
def contains_rejection_reason(rejection_text, reason):
    try:
        if rejection_text and isinstance(rejection_text, str):
            return reason.lower() in rejection_text.lower()
    except Exception as e:
        handle_error(f"Error in contains_rejection_reason: {str(e)}")
        return False
    return False

# Fixed function to map rejection reasons
def map_rejection_reason(rejection_text):
    try:
        if rejection_text and isinstance(rejection_text, str):
            for reason, rejection_class in REJECTION_REASONS_MAP.items():
                if contains_rejection_reason(rejection_text, reason):
                    return rejection_class
            return "Unknown"
        else:
            return "NoRemark"
    except Exception as e:
        handle_error(f"Error in map_rejection_reason: {str(e)}")
        return "Error"

# Fixed complex rejection classifier function
def complex_rejection_classifier(remark_text):
    try:
        if not isinstance(remark_text, str) or len(remark_text.strip()) == 0:
            return "Invalid Remark"

        # Check 
        fake_doc = contains_rejection_reason(remark_text, "Fake_document")
        not_covered = contains_rejection_reason(remark_text, "Not_Covered")
        policy_expired = contains_rejection_reason(remark_text, "Policy_expired")

        if fake_doc:
            return "Fake_document"
        elif not_covered:
            return "Not_Covered"
        elif policy_expired:
            return "Policy_expired"
        else:
            # Unknown or null remarks
            return map_rejection_reason(remark_text)
    except Exception as e:
        handle_error(f"Error in complex_rejection_classifier: {str(e)}")
        return "Error"

def apply_rejection_classifier(data):
    """Apply the fixed rejection classifier to the dataset"""
    if not data or len(data) <= 1:
        return None
    
    
    header = data[0]
    columns = {name: idx for idx, name in enumerate(header)}
    
    # Add new column for rejection class
    header.append('REJECTION_CLASS')
    
    
    global REJECTION_REASONS_MAP
    REJECTION_REASONS_MAP = {
        "Fake_document": "Fake_document",
        "Not_Covered": "Not_Covered",
        "Policy_expired": "Policy_expired"
    }
    
    
    for i in range(1, len(data)):
        row = data[i]
        
        
        remarks_idx = columns.get('REJECTION_REMARKS')
        if remarks_idx is not None and remarks_idx < len(row):
            remarks = row[remarks_idx]
            if remarks:
                rejection_class = complex_rejection_classifier(remarks)
            else:
                rejection_class = "No Remark"
        else:
            rejection_class = "No Remark"
        
        
        row.append(rejection_class)
    
    return data

In [4]:
def generate_visualizations(city_stats):
    """Generate visualizations for city analysis"""
    
    import matplotlib.pyplot as plt
    
    if not city_stats:
        print("No data available for visualization")
        return
    
    # Filter to only consider the four cities mentioned in the requirements
    target_cities = ['PUNE', 'KOLKATA', 'RANCHI', 'GUWAHATI']
    filtered_stats = {city: stats for city, stats in city_stats.items() if city in target_cities}
    
    if not filtered_stats:
        print("No target cities found in data")
        return
    
    # Data preparation
    cities = list(filtered_stats.keys())
    claim_amounts = [filtered_stats[city]['total_claim_amount'] for city in cities]
    premium_amounts = [filtered_stats[city]['total_premium'] for city in cities]
    paid_amounts = [filtered_stats[city]['total_paid'] for city in cities]
    loss_ratios = [filtered_stats[city]['loss_ratio'] for city in cities]
    
    # Plot 1: Claims, Premiums, and Paid Amounts by City
    plt.figure(figsize=(12, 6))
    
    x = range(len(cities))
    width = 0.25
    
    plt.bar([i - width for i in x], claim_amounts, width, label='CLAIM_AMOUNT')
    plt.bar(x, premium_amounts, width, label='PREMIUM_COLLECTED')
    plt.bar([i + width for i in x], paid_amounts, width, label='PAID_AMOUNT')
    
    plt.ylabel('Amount')
    plt.title('Claims, Premiums, and Paid Amounts by City (April 2025)')
    plt.xticks(x, cities)
    plt.legend()
    
    
    plt.tight_layout()
    plt.savefig('claims_by_city.png')
    plt.close()
    
    # Plot 2: Loss Ratio by City
    plt.figure(figsize=(10, 6))
    
    plt.bar(cities, loss_ratios)
    plt.ylabel('Loss Ratio')
    plt.title('Loss Ratio by City (April 2025)')
    
    
    plt.tight_layout()
    plt.savefig('loss_ratio_by_city.png')
    plt.close()
    
    # Plot 3: Profit/Loss by City
    plt.figure(figsize=(10, 6))
    
    profits = [filtered_stats[city]['profit'] for city in cities]
    colors = ['green' if p > 0 else 'red' for p in profits]
    
    plt.bar(cities, profits, color=colors)
    plt.ylabel('Profit/Loss (₹)')
    plt.title('Profit/Loss by City (April 2025)')
    plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    
    
    plt.tight_layout()
    plt.savefig('profit_by_city.png')
    plt.close()
    
    # Plot 4: Rejection Rate by City
    plt.figure(figsize=(10, 6))
    
    rejection_rates = [filtered_stats[city]['rejection_rate'] * 100 for city in cities]
    
    plt.bar(cities, rejection_rates)
    plt.ylabel('Rejection Rate (%)')
    plt.title('Claim Rejection Rate by City (April 2025)')
    
    
    plt.tight_layout()
    plt.savefig('rejection_rate_by_city.png')
    plt.close()
    
    print("Visualizations have been generated and saved as PNG files.")
    
    return True


def main():
    
    cleaned_data = clean_and_preprocess_csv('Insurance_auto_data.csv')
    
    if not cleaned_data:
        print("Failed to process the CSV file.")
        return
    
    
    classified_data = apply_rejection_classifier(cleaned_data)
    
    if not classified_data:
        print("Failed rj classifier.")
        return
    
    
    city_stats = get_city_statistics(classified_data)
    
    if not city_stats:
        print("Failed to calculate city statistics.")
        return
    
    
    try:
        generate_visualizations(city_stats)
    except ImportError:
        print("visualgen error")
    
    # Recommend city for closure
    recommendation, detailed_stats = recommend_city_closure(city_stats)
    
    
    print("\nCITY ANALYSIS RESULTS")
    print(f"Recommended city for closure: {recommendation}")
    print("\nDetailed statistics by city:")
    
    for city, stats in detailed_stats.items():
        print(f"\n{city}:")
        print(f"  Total Claims: {stats['total_claims']}")
        print(f"  Total Claim Amount: ₹{stats['total_claim_amount']:.2f}")
        print(f"  Total Premium Collected: ₹{stats['total_premium']:.2f}")
        print(f"  Total Paid Amount: ₹{stats['total_paid']:.2f}")
        print(f"  Rejected Claims: {stats['rejected_claims']}")
        print(f"  Rejection Rate: {stats['rejection_rate']:.2%}")
        print(f"  Profit/Loss: ₹{stats['profit']:.2f}")
        print(f"  Loss Ratio: {stats['loss_ratio']:.2f}")
    
    print("\n----- SAMPLE OF PROCESSED DATA WITH REJECTION CLASSIFICATIONS -----")
    # Print header
    print(','.join(classified_data[0]))
    
    # Print first few rows of processed data
    for i in range(1, min(6, len(classified_data))):
        row_str = []
        for cell in classified_data[i]:
            if isinstance(cell, (int, float)):
                row_str.append(str(cell))
            else:
                row_str.append(cell)
        print(','.join(row_str))

if __name__ == "__main__":
    main()

Visualizations have been generated and saved as PNG files.

CITY ANALYSIS RESULTS
Recommended city for closure: KOLKATA

Detailed statistics by city:

PUNE:
  Total Claims: 37
  Total Claim Amount: ₹1543919.00
  Total Premium Collected: ₹369254.79
  Total Paid Amount: ₹1093273.28
  Rejected Claims: 3
  Rejection Rate: 8.11%
  Profit/Loss: ₹-724018.49
  Loss Ratio: 2.96

GUWAHATI:
  Total Claims: 24
  Total Claim Amount: ₹1148030.00
  Total Premium Collected: ₹261314.84
  Total Paid Amount: ₹753189.43
  Rejected Claims: 3
  Rejection Rate: 12.50%
  Profit/Loss: ₹-491874.59
  Loss Ratio: 2.88

RANCHI:
  Total Claims: 17
  Total Claim Amount: ₹570328.00
  Total Premium Collected: ₹148858.60
  Total Paid Amount: ₹401142.67
  Rejected Claims: 2
  Rejection Rate: 11.76%
  Profit/Loss: ₹-252284.07
  Loss Ratio: 2.69

KOLKATA:
  Total Claims: 16
  Total Claim Amount: ₹847893.00
  Total Premium Collected: ₹140279.78
  Total Paid Amount: ₹636392.57
  Rejected Claims: 0
  Rejection Rate: 0.00%
  